# Proyecto

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz


### Equipo:

- Sebastián Tinoco
- Samuel Molina
- Usuario codalab: stinoco
- Equipo: renacín te paseo

### Link de repositorio de GitHub: 
* Samuel → `https://github.com/samumolina/evaluaciones_mds7202`
* Sebastián → `https://github.com/sebatinoco/MDS7202`


## 0. Librerías

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
path = '/content/drive/MyDrive/MDS/Labs/Competencia/'
#path = '/content/drive/MyDrive/Competencia/'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from itertools import combinations, permutations

from numpy import inf

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, r2_score
from sklearn.manifold import TSNE
from sklearn.utils import compute_sample_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.dummy import DummyClassifier, DummyRegressor

import torch # para chequear si tenemos gpu xd

!pip install umap-learn
import umap

import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# si gpu esta disponible, habilitamos lightgbm gpu
if torch.cuda.is_available():
  !git clone --recursive https://github.com/Microsoft/LightGBM
  !cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
  !pip install cmake

import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor

from xgboost import XGBRegressor, XGBClassifier

!pip install optuna
import optuna
from optuna.integration import LightGBMPruningCallback

from copy import deepcopy

import warnings
warnings.filterwarnings("ignore")

import pickle 

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.1 MB/s 
     |████████████████████████████████| 1.1 MB 34.0 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=16e456b26f83d01325988e362cc8ba3ac54a47c639ade71669a8f2046097a739
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=ba2eaf3f1ce54da182a71a9a9892fb6600f4f01f165c3e79066f88b0fce2f4c3
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 8.7 MB/s 
 


## 1. Introducción

El objetivo de este proyecto consiste en generar modelos predectivos para Renacín, famoso corpóreo de la localidad de Maipú. En términos más concretos, se busca predecir tanto los ingresos como la calidad de una película, lo cual es trabajado a través de 2 modelos distintos: uno de **clasificación** y otro de **regresión**.

Los datos que proveen es un dataset con 9.641 ejemplos que describen las principales características de una película en particular.
Son en total, inicialmente, 18 atributos entre variables numéricas, categóricas y de texto. Las variables objetivos son de tipo categórico para la tarea de clasificación y de tipo numérica para el caso de la regresión.

Para llevar a cabo ambas tareas, los datos se limpiaron y posteriormente se codificaron y normalizaron según el tipo de variable. Adicionalmente se generaron features que pudieran entregar más (y mejor) información para los distintos problemas.

La tarea de clasificación se evalua en base a la métrica **f1-macro** ya que se necesita entender cómo se comporta el clasificador para cada una de las clases y esta permite medir una ponderación entre Presicion y Recall. Para el caso de la tarea de regresión, la métrica de evaluación será **r2** la cual permite determinar si la variables independientes utilizadas permiten explicar la variable target, midiendo el nivel de correlación entre la variable real y la predicha.

Nuestra propuesta para resolver ambas tareas consistió en la optimización de la métrica escogida (`f1` o `r2`) a través de los siguientes componentes:
1. Pre procesamiento: Fijando un clasificador (`LGBMClassifier`) y un regresor (`LGBMRegressor`), se prueban diferentes configuraciones del pre procesamiento para luego escoger la que obtiene mejores resultados en la tarea a resolver. 
2. Hiperparámetros del clasificador: Usando la configuración óptima escogida en el paso anterior, se optimizan los hiperparámetros de los clasificadores mediante `HalvingGridSearchCV`.

Si bien es posible optimizar ambos pasos al mismo tiempo mediante `GridSearchCV` y un `Pipeline` "end-to-end" que consolide todo lo anterior, realizar esto no es óptimo por las siguientes razones:
1. Para obtener conclusiones válidas, `GridSearchCV` necesita generar validaciones cruzadas sobre cada combinación de hiperparámetros. Esto quiere decir que, para cada combinacion, se debe probar `n` veces (número que usualmente va entre 5 y 10).
2. Considerando que se debe buscar la configuración óptima tanto en pre procesamiento como en los hiperparámetros, el campo de espacio de búsqueda crece de manera exponencial.
3. En vista de todo lo anterior y recordando que un `Pipeline` "end-to-end" requiere un tiempo de entrenamiento no menor (45 a 60 segundos), usar `GridSearchCV` para optimizar ambos pasos a la misma vez se hace computacionalmente intratable (con tiempos cercanos a 1/3 de la edad del universo jaja).

Tras la optimización, se llegó que la mejor configuración de pre procesamiento fue con Bag of Words en el caso del problema de clasificación y TFIDF para el problema de regresión. Además, en ambos problemas se ocuparon features creadas (**Feature Engineering**) y una selección póstuma de los atributos usando `SelectPercentile`.

Finalmente, los mejores resultados obtenidos en el conjunto de test luego de la optimización de cada uno de los modelos fue de **f1 = 0.33** para el caso de la clasificación y **r2 = 0.61** para la regresión. Estos resultados son adecuados para considerar los modelos como buenos clasificadores/predictores. En relación a la competencia en Codalab, se superó el resultado base en clasificación (logrando un segundo lugar al momento de escribir este informe), pero no en regresión (8vo lugar).

---
## 2. Preparación y Análisis Exploratorio de Datos

In [ ]:
##  Código Preparación de Datos.

# leemos features
numerical_features = pd.read_parquet(path + 'train_numerical_features.parquet') # features numericas
numerical_features = numerical_features.drop(columns = ['title', 'tagline', 'credits'])
print(f"Dimensiones features numéricas: {numerical_features.shape}")
text_features = pd.read_parquet(path + 'train_text_features.parquet') # features de texto
print(f"Dimensiones features de texto: {text_features.shape}")

# mergeamos datasets
df = numerical_features.merge(text_features, on = 'id')
print(f'Dimensiones merge: {df.shape}')

df = df.drop(columns = ['poster_path', 'backdrop_path', 'recommendations']) # drop de columnas
df = df[df['revenue'] != 0] # filtrar ejemplos con revenue = 0
df[~(pd.isnull(df['release_date'])) & ~(pd.isnull(df['runtime']))] # drop si release_date y run_time son nulos
df['release_date'] = pd.to_datetime(df['release_date']) # pasamos release_date a datetime
df = df[df['status'] == 'Released'] # conservamos las filas donde status = Released

text_columns = list(df.select_dtypes('object').columns) # features de texto
df[text_columns] = df.select_dtypes('object').fillna('') # rellenamos columnas string con ''
df['label'] = pd.cut(df['vote_average'], bins = [0, 5, 6, 7, 8, 10], # generamos categorias
              labels = ['Negative', 'Mixed', 'Mostly Positive', 'Positive', 'Very Positive'])

df = df.drop(columns = ['vote_average', 'id']) # drop de vote_average y id
df = df.rename(columns = {'revenue': 'target'}) # rename revenue -> target
df = df.reset_index(drop = True)
df

Dimensiones features numéricas: (9641, 8)
Dimensiones features de texto: (9641, 11)
Dimensiones merge: (9641, 18)


,budget,target,runtime,status,title,genres,original_language,overview,production_companies,release_date,tagline,credits,keywords,label
0,200000000.0,400000000.0,142.0,Released,Fantastic Beasts: The Secrets of Dumbledore,Fantasy-Adventure-Action,en,Professor Albus Dumbledore knows the powerful ...,Warner Bros. Pictures-Heyday Films,2022-04-06,Return to the magic.,Jude Law-Eddie Redmayne-Mads Mikkelsen-Ezra Mi...,magic-curse-fantasy world-wizard-magical creat...,Mostly Positive
1,110000000.0,393000000.0,122.0,Released,Sonic the Hedgehog 2,Action-Adventure-Family-Comedy,en,After settling in Green Hills Sonic is eager t...,Original Film-Blur Studio-Marza Animation Plan...,2022-03-30,Welcome to the next level.,James Marsden-Ben Schwartz-Tika Sumpter-Natash...,sequel-based on video game-hedgehog-live actio...,Positive
2,74000000.0,164289828.0,112.0,Released,The Lost City,Action-Adventure-Comedy,en,A reclusive romance novelist was sure nothing ...,Paramount-Fortis Films-3dot Productions-Exhibi...,2022-03-24,The adventure is real. The heroes are not.,Sandra Bullock-Channing Tatum-Daniel Radcliffe...,duringcreditsstinger,Mostly Positive
3,75000000.0,161000000.0,105.0,Released,Morbius,Action-Science Fiction-Fantasy,en,Dangerously ill with a rare blood disorder and...,Columbia Pictures-Avi Arad Productions-Matt To...,2022-03-30,A new Marvel legend arrives.,Jared Leto-Matt Smith-Adria Arjona-Jared Harri...,vampire-based on comic,Mostly Positive
4,120000000.0,400780000.0,116.0,Released,Uncharted,Action-Adventure,en,A young street-smart Nathan Drake and his wise...,Columbia Pictures-Atlas Entertainment-PlayStat...,2022-02-10,Fortune favors the bold.,Tom Holland-Mark Wahlberg-Sophia Ali-Tati Gabr...,treasure-treasure hunt-based on video game-dlb,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,36000000.0,44229441.0,155.0,Released,Amistad,Drama-History-Mystery,en,In 1839 the slave ship Amistad set sail from C...,DreamWorks Pictures,1997-12-10,Freedom is not given. It is our right at birth...,Morgan Freeman-Nigel Hawthorne-Anthony Hopkins...,cuba-mutiny-slavery-sentence-historical figure...,Mostly Positive
6447,18000000.0,476684675.0,103.0,Released,Home Alone,Comedy-Family,en,Eight-year-old Kevin McCallister makes the mos...,Hughes Entertainment-20th Century Fox,1990-11-16,A family comedy without the family.,Macaulay Culkin-Joe Pesci-Daniel Stern-John He...,holiday-burglar-slapstick-little boy-family re...,Positive
6448,0.0,3967001.0,100.0,Released,Ip Man: The Final Fight,Action-Drama,cn,In postwar Hong Kong legendary Wing Chun grand...,Emperor Motion Pictures-Cinemasia-National Art...,2013-03-22,,Anthony Wong-Anita Yuen-Gillian Chung-Jordan C...,biography,Mostly Positive
6449,25000000.0,23800000.0,92.0,Released,A Rainy Day in New York,Comedy-Romance,en,Two young people arrive in New York to spend a...,Gravier Productions-Perdido Productions-FilmNa...,2019-07-26,Love In Spring.,Timothée Chalamet-Elle Fanning-Selena Gomez-Ju...,new york city,Mostly Positive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6451 entries, 0 to 6450
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                6451 non-null   float64       
 1   target                6451 non-null   float64       
 2   runtime               6451 non-null   float64       
 3   status                6451 non-null   object        
 4   title                 6451 non-null   object        
 5   genres                6451 non-null   object        
 6   original_language     6451 non-null   object        
 7   overview              6451 non-null   object        
 8   production_companies  6451 non-null   object        
 9   release_date          6451 non-null   datetime64[ns]
 10  tagline               6451 non-null   object        
 11  credits               6451 non-null   object        
 12  keywords              6451 non-null   object        
 13  label             

El dataset contiene 9 features de tipo `object` que son cadenas de texto, 3 variables numéricas, 1 variable categórica y 1 variable temporal.

In [ ]:
df.isna().sum()

budget                  0
target                  0
runtime                 0
status                  0
title                   0
genres                  0
original_language       0
overview                0
production_companies    0
release_date            0
tagline                 0
credits                 0
keywords                0
label                   0
dtype: int64

El dataset no tiene valores faltantes.

In [ ]:
df_obj = df.select_dtypes(include=["object"])

for i in df_obj.columns:
    print(f'La variable {i} tiene {len(df[i].unique())} valores distintos')

La variable status tiene 1 valores distintos
La variable title tiene 6451 valores distintos
La variable genres tiene 1574 valores distintos
La variable original_language tiene 34 valores distintos
La variable overview tiene 6449 valores distintos
La variable production_companies tiene 5621 valores distintos
La variable tagline tiene 5685 valores distintos
La variable credits tiene 6450 valores distintos
La variable keywords tiene 6243 valores distintos


In [ ]:
# Variables categóricas
df_cat = df.select_dtypes(include=["category"])

for i in df_cat.columns:
    fig = px.histogram(df_cat, x=i,title="Distribución de variable "+i)
    fig.show()

Clases `Very Positive` y `Negative` tienen considerablemente menos muestras que el resto de las clases. Esto implica que tendremos problemas de desbalance de clases que deberán ser tratados a la hora de entrenar el modelo.

In [ ]:
# Variable object original_language
fig = px.histogram(df, x='original_language',title="Distribución de variable original_language")
fig.update_xaxes({'categoryorder':'total descending'})
fig.show()

Lenguaje original más repetido es "en" → English.

In [ ]:
# Variables numéricas
df_num = df.select_dtypes(exclude=["category","object"])

for i in df_num.columns:
    fig = px.histogram(df_num, x=i, marginal="box",title="Distribución de variable "+i)
    fig.show()

In [ ]:
# Graficamos variables en logaritmo
for col in ['budget', 'target']:
  fig = px.histogram(df_num, x='target', marginal="box",title=f"Distribución logarítmica de la variable {col}", log_y = True)
  fig.show()

Se aprecia que la variable `runtime` correspondiente a la duración de la película tiene una distribución visualmente gaussiana.

En relación a la variable `release date`, se observa que desde 1980 aumentaron considerablemente los lanzamientos de películas. 

Para mejor visualización se graficó también la distribución logarítima de las variables `budget` y `target`, las que recuerdan a una distribución Chi2.

In [ ]:
# Interacciones entre variables
fig = px.scatter_matrix(
    df_num,
    height=800,
    title="Scatter matrix")

fig.update_traces(diagonal_visible=False, showupperhalf=False)
fig.show()

En general, se observa una correlación positiva en todas las variables. Un caso interesante es la dinámica del par de variables `release_date` y `runtime`, pues los datos parecen estar concentrados en las mismas coordenadas a través de `release_date`. Esto quiere decir que, para toda fecha registrada del dataset, el tiempo de la película se mantiene relativamente estable (entre 100 y 150 minutos).

In [ ]:
# Correlación entre variables numéricas

correlacion = df_num.iloc[:,:3].corr()

fig=px.imshow(correlacion,
          labels=dict(x="", y="", color="correlacion"),
          x=df_num.iloc[:,:3].columns,
          y=df_num.iloc[:,:3].columns,
          zmin=-1,
          zmax=1,
          )

fig.show()

Variables `target` y `budget` están altamente correlacionadas, teniendo una correlación positiva y superior a 0.7 (en otras palabras, a mayor presupuesto → mayores ganancias.)

Se aprecia también una correlación positiva pero de menor magnitud entre `runtime` y `budget`.

In [ ]:
px.scatter(df, 
       x='budget', 
       y='target', 
       title = 'Scatterplot de budget vs target')

In [ ]:
# Preprocesamiento básico de variables numéricas

# Columns transformer
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('StandardScaler', StandardScaler(),
         df_num.iloc[:,:3].columns),
    ])

#Pipelines

pipe_process_2d_umap = Pipeline(
    [("Preprocesamiento", preprocessing_transformer),("Reducción de dimensionalidad",
                                                      umap.UMAP(n_components=2, random_state=42))]
)

data_process_2d_umap=pd.DataFrame(pipe_process_2d_umap.fit_transform(df),
                                  columns=['Dimensión 1','Dimensión 2'])

data_process_2d_umap['label'] = df['label']

px.scatter(data_process_2d_umap, x= 'Dimensión 1', y= 'Dimensión 2', color = 'label')

  De acuerdo a la proyección UMAP mostrada en el gráfico anterior, se observa que las variables numéricas utilizadas en la generación de este no permiten separar de forma clara la distintas clases.

In [ ]:
# Películas de Marvel

## boxplot
df_marvel = df.copy()
df_marvel['is_marvel'] = np.where(df['production_companies'].str.contains('Marvel ', regex = False, na = False), 'Película de Marvel', 'Otras películas')

for col in ['budget', 'target']:
  fig = px.box(df_marvel, y=col, color = 'is_marvel', title = f'Boxplot de {col}: películas Marvel vs otras películas')
  fig.show()


## scatter plot
marvel_movies = df[df['production_companies'].str.contains('Marvel ', regex=False, na=False)]
fig = px.scatter(marvel_movies, 
       x='budget', 
       y='target',
       hover_name='title',
       title = 'Scatterplot de budget vs target en películas de Marvel')

fig.show()

* De los gráficos de caja, se observa que las películas de Marvel tienen tanto un mayor presupuesto (`budget`) como mejores ganancias (`target`).
* Por último y en términos generales, las películas Marvel que mayor presupuesto tuvieron, mayores ganancias generaron. Una situación particular se observa en las películas que tuvieron 200M de presupuesto. *Black Widow* y *Spider-Man: No Way Home* tuvieron el mismo presupuesto, pero Spiderman generó 1.892B y Black Widow 379.75M.

A continuación se define la función top_n_mas_frecuentes que permite obtener el top n de cualquier feature categórica.

In [ ]:
def top_n_mas_frecuentes(feature, n):
    
    data = []
    
    tokens = df[feature].apply(lambda x: x.split('-'))
    
    for token in tokens:
        data.append(token)
    
    all_tokens = [] #guarda tokens
    for i in range(len(data)): #ciclo  que une listas de tokens
        all_tokens += data[i]
        
    df_ = pd.DataFrame(pd.DataFrame(all_tokens)[0].value_counts()[1:n+1]).reset_index()
    df_.columns = [feature,'frecuencia']
    
    fig = px.bar(df_, x=feature, y='frecuencia', title='Top '+str(n)+' '+feature)
    return df_, fig 

In [ ]:
for feature, count in {'production_companies': 50, 'keywords': 50, 'credits': 100, 'genres': 15}.items():
  a, b = top_n_mas_frecuentes(feature, count)
  b.show()

A continuación, se presenta el comportamiento de las clases frente `budget` y `target` (si bien no podemos usar esta última, es interesante analizar la percepción del público con las ganancias recibidas).

In [ ]:
for col in ['budget', 'target']:
  fig = px.box(df, y=col, color = 'label', title = f'Boxplot de {col} separado por la evaluación de la película (label)')
  fig.show()

En vista de los gráficos anteriores, no se observan mayores diferencias entre las clases en torno al presupuesto y ganancias recibidas. 

Podemos ir un paso mas allá y visualizar este análisis a través del tiempo:

In [ ]:
def time_graph(feature):
  budget_x_label = df[['release_date', feature, 'label']]

  all_dfs = []
  for label in list(df['label'].unique()):
      all_dfs.append(budget_x_label[budget_x_label['label']==label])

  all_dfs_v2 = []
  for df_ in all_dfs:
      df_ = df_.copy()
      df_.set_index('release_date', inplace = True)    
      clase = df_['label'][0]
      df_ = df_.drop(columns = ['label'])
      df_ = pd.DataFrame(df_[feature].resample('Y').mean())
      df_ = df_.fillna(0)
      
      labels = []
      for i in range(df_.shape[0]):
          nombre = clase
          labels.append(nombre)
          
      df_['label'] = labels
      # df_['release_date'] = df_.index
      df_.reset_index(inplace=True, drop=False)
      
      all_dfs_v2.append(df_)

  concat_df = pd.concat([all_dfs_v2[0], all_dfs_v2[1], all_dfs_v2[2], all_dfs_v2[3], all_dfs_v2[4]], axis=0)
  concat_df = concat_df.reset_index(inplace=False, drop=False)

  # gráfico
  fig = px.line(concat_df, 
          x='release_date', 
          y=feature, 
          color='label',
          title=f'{feature.title()} promedio por año de cada clase')

  fig.show()

time_graph('budget')
time_graph('target')

Se concluye que, en general,todas las clases tienen una trayectoria relativamente similar entre sí. En otras palabras, el tiempo no parece ser una variable crítica para separar las clases. Aún así, sobresale el caso de la clase `Very Positive`, la cual tiene un gran salto en su presupuesto para el 2008. 

---

## 3. Preprocesamiento, Holdout y Feature Engineering

### Funciones y Objetos Auxiliares

La creación de funciones auxiliares tienen como próposito facilitar la ejecución de distintas pruebas para cada una de las tareas a resolver. La idea es que los modelamientos tanto de clasificación como regresión sigan una línea similar de acuerdo a sus características. También se definieron distintas nuevas features que se consideraron relevantes para poder explicar tanto `label` para la clasificación como `target` para la regresión.

Para el resto del informe y con el objetivo de ordenar el código siguiente, se harán uso de las siguientes funciones y objetos:

In [ ]:
features = df.copy()
targets = df[['target', 'label']].copy()
features.drop(['target', 'label','title', 'status'],axis=1,inplace=True)

def is_on(list1, list2):
    '''
    Función que recibe dos listas distintas 
    recorre la primera y cuenta cuantos elementos
    de esta están en la segunda lista.
    Retorna el conteo.
    '''
    count = 0
    for i in list1:
        if i in list2:
            count = count + 1
    return count        

En el problema de la clasificación, puede ser útil resamplear la muestra obtenida, por lo que haremos uso de la siguiente función:

In [ ]:
def resample(X, y, param_over = False, param_under = False, random_state = 3380):

    '''
    Función que recibe dataframes de entrada y los devuelve resampleados.
    X: dataframe de entrada (X_train)
    y: dataframe de entrada (y_train)
    param_over: factor por el que se hará oversampling
    param_under: factor por el que se hará subsampling
    '''

    
    target_classes = ['Very Positive', 'Negative', 'Mixed'] # clases problemáticas
    
    # Oversampling
    if param_over:
        count_values = y.value_counts()
        dict_over = {key: int(count_values[key] * (1 + param_over)) for key in count_values.keys() if key in target_classes}

        if X.ndim == 1:
            X = np.array(X).reshape(-1, 1)
        oversampler = RandomOverSampler(sampling_strategy = dict_over, random_state = random_state)
        X, y = oversampler.fit_resample(X, y)

    # Subsampling
    if param_under:
        count_values = y.value_counts()
        dict_under = {key: int(count_values[key] * (param_under)) for key in count_values.keys() if key not in target_classes}
        if X.ndim == 1:
            X = np.array(X).reshape(-1, 1)

        undersampler = RandomUnderSampler(sampling_strategy = dict_under, random_state = random_state)
        X, y = undersampler.fit_resample(X, y)

    return X, y

A continuación, se implementan los vectorizadores `CountVectorizer` (Bag of Words) y `TfidfVectorizer` (TFIDF) para luego ser usados en los experimentos:

In [ ]:
# analyzer para separar creditos
def custom_tokenizer(text):
    return text.split('-')

# Tokenizer para Stemizar
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Vectorizador BoW
bow_credits = CountVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 4), min_df = 5)
bow_genres = CountVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 3), min_df = 5)
bow_companies = CountVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 3), min_df = 5)
bow_keywords = CountVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 2), min_df = 5, max_df = .8)
bow_overview = CountVectorizer(tokenizer = StemmerTokenizer(), ngram_range = (1, 3), min_df = 5, max_df = .8)

bow_dict = {'credits': bow_credits, 'genres': bow_genres, 
            'production_companies': bow_companies, 'keywords': bow_keywords, 'overview': bow_overview}

# Vectorizador TFIDF
tfidf_credits = TfidfVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 4), min_df = 5)
tfidf_genres = TfidfVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 3), min_df = 5)
tfidf_companies = TfidfVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 3), min_df = 5)
tfidf_keywords = TfidfVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 2), min_df = 5, max_df = .8)
tfidf_overview = TfidfVectorizer(tokenizer = StemmerTokenizer(), ngram_range = (1, 3), min_df = 5, max_df = .8)

tfidf_dict = {'credits': tfidf_credits, 'genres': tfidf_genres, 
            'production_companies': tfidf_companies, 'keywords': tfidf_keywords, 'overview': tfidf_overview}

# agrupamos ambos en un diccionario
vectorizer_dict = {'bow': bow_dict, 'tfidf': tfidf_dict}

### Holdout

Para hacer el *Holdout*, se hará uso de la función `train_model`, la cual tiene el objetivo de separar los datos y entrenar el modelo del experimento. 

En cuanto al **Holdout**, se aprecia lo siguiente:

1. Para el caso de la clasificación y con tal de tratar el desbalance de clases, se usa el parámetro `stratify` y así asegurar la misma distribución de clases entre conjunto de entrenamiento y testeo.
2. Se separa la data en 75% para entrenamiento y 25% para testeo.

In [ ]:
## Código Holdout

X = df.drop(columns = ['label', 'target'])

def train_model(pipeline, mode, param_under = None, param_over = None, fit_weights = False, random_state = 3380):
    '''
    Función que recibe un pipeline y lo entrena. Depende también del problema (mode) a resolver (clasificación: clf, regresión: reg).
    pipeline: Objeto Pipeline a ser entrenado
    mode: Problema a resolver ("clf" o "reg", str)
    param_over: Factor por el que se hará oversampling (float)
    param_under: Factor por el que se hará subsampling (float)
    fit_weights: Indica si queremos usar los "factores de expansión" de la muestra. Puede favorecer el desbalance de clases (bool)
    '''
    
    # si el problema es de clasificación
    if mode == 'clf':

      # dividimos datos usando estratificación
      y = df['label'].copy()
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, shuffle = True, stratify = y, random_state = random_state)

      # podemos especificar under y over sampling
      if (param_under is not None) | (param_over is not None):
        X_train, y_train = resample(X_train, y_train, param_over = param_over, param_under = param_under, random_state = random_state)
    
      # podemos agregar factores de expansión
      if fit_weights:
        sample_weight = compute_sample_weight(class_weight = 'balanced', y = y_train)
        pipeline.fit(X_train, y_train, clf__sample_weight = sample_weight)

      # entrenamos pipeline y obtenemos predicciones
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)

      # print de métricas
      print(f"F1: {f1_score(y_test, y_pred, average = 'macro'):.3f}")
      print(classification_report(y_test, y_pred))

    # si el problema es de regresión
    elif mode == 'reg':
      
      # dividimos datos
      y = df['target'].copy()
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, shuffle = True, random_state = 3380)

      # entrenamos pipeline y obtenemos predicciones
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)

      # print de métricas
      print(f"R2: {r2_score(y_test, y_pred):.3f}")

    # pelmazo xd
    else:
      raise ValueError('Debes especificar tu problema a modelar, pelmazo!')

### Feature Engineering

A continuación, se presentan las features creadas a través de *Feature Engineering*:

In [ ]:
###### FEATURE ENGINEERING #######

df_rank, b = top_n_mas_frecuentes('production_companies', 15)
production_rank = list(df_rank['production_companies'])

df_rank, b = top_n_mas_frecuentes('credits', 30)
credit_rank = list(df_rank['credits'])

df_rank, b = top_n_mas_frecuentes('keywords', 50)
key_rank = list(df_rank['keywords'])

def custom_features(df):

    df_base = df.copy()
    df_base['year'] = df['release_date'].apply(lambda x: x.year)

    # dataframe con features a retornar
    df_tmp = pd.DataFrame()

    df_tmp['year'] = df['release_date'].apply(lambda x: x.year)
    df_tmp['period'] = pd.cut(df_tmp['year'], bins = [0, 1980, 2000, 2050]) # categorica con el periodo
    df_tmp['is_2020'] = np.where(df_tmp['year'] == 2020, 1, 0)
    df_tmp['month'] = df['release_date'].apply(lambda x: x.month).astype('category')
    df_tmp['week'] = pd.cut(df['release_date'].apply(lambda x: x.day), bins = [0, 7, 15, 21, 32])
    df_tmp['budget_to_runtime'] = df['budget'] / df['runtime']
    df_tmp['budget_year_ratio'] = df_base['budget'] / (df_base['year'] * df_base['year'])
    df_tmp['runtime_to_mean_year'] = df_base['runtime'] / df_base.groupby("year")["runtime"].transform('mean')
    df_tmp['budget_to_mean_year'] = df_base['budget'] / df_base.groupby("year")["budget"].transform('mean')
    df_tmp['budget_to_year'] = df_base['budget'] / df_base['year']
    df_tmp['budget_to_mean_year_to_year'] = df_tmp['budget_to_mean_year'] / df_tmp['year']
    df_tmp['log_budget'] = np.log(df['budget'])
    df_tmp['year_to_log_budget'] = df_tmp['year'] / df_tmp['log_budget']
    df_tmp['budget_to_runtime_to_year'] = df_tmp['budget_to_runtime'] / df_tmp['year']
    df_tmp['inflationBudget'] = df_base['budget'] + df_base['budget']*1.8/100*(2018-df_base['year'])

    # conteos → cuántos géneros, cuántos actores, cuántas productoras
    df_tmp['count_genres'] = pd.DataFrame(features['genres'].apply(lambda x: x.split('-')))['genres'].apply(lambda x: len(x))
    df_tmp['count_credits'] = pd.DataFrame(features['credits'].apply(lambda x: x.split('-')))['credits'].apply(lambda x: len(x))
    df_tmp['count_production'] = pd.DataFrame(features['production_companies'].apply(lambda x: x.split('-')))['production_companies'].apply(lambda x: len(x))
    
    # tiene una de las 10 productoras más famosas
    df_tmp['have_top_prod'] = pd.DataFrame(features['production_companies'].apply(lambda x: x.split('-')))['production_companies'].apply(lambda x: 1 if is_on(production_rank,x)!=0 else 0 )
    
    # tiene uno de las 30 actores más famosas
    df_tmp['have_top_actor'] = pd.DataFrame(features['credits'].apply(lambda x: x.split('-')))['credits'].apply(lambda x: 1 if is_on(credit_rank,x)!=0 else 0 )
    
    # tiene uno de las 50 keywords más famosas
    df_tmp['have_top_key'] = pd.DataFrame(features['keywords'].apply(lambda x: x.split('-')))['keywords'].apply(lambda x: 1 if is_on(key_rank,x)!=0 else 0 )
    
    # es una pelicula de marvel
    df_tmp['is_marvel'] = np.where(df['production_companies'].str.contains('Marvel ', regex = False, na = False), 1, 0)

    # duración película
    df_tmp['duracion'] = pd.cut(features['runtime'], bins = [0, 40, 110, 400], # generamos categorias
              labels = ['Low', 'Medium', 'Long'])
    
    df_tmp = df_tmp.replace(np.nan, 0) # para todos los ratios, rellenamos con 0s valores nulos
    df_tmp = df_tmp.replace(np.inf, 0) # para todos los ratios, rellenamos con 0s valores nulos
    df_tmp = df_tmp.replace(-np.inf, 0) # para todos los ratios, rellenamos con 0s valores nulos

    return df_tmp

ct_dates = ColumnTransformer([
    ('year', RobustScaler(), ['year']), # estandarizamos año
    ('month', OneHotEncoder(handle_unknown = 'ignore'), ['month', 'period', 'week', 'duracion'])
], sparse_threshold = 0, remainder = 'passthrough') # devolvemos datos "dense" y pasamos todas las features

fun_transformer = Pipeline([
    ('features', FunctionTransformer(custom_features)), # genero features
    ('onehot_dates', ct_dates) # transformo año y mes
])

# vista de features generadas
results = fun_transformer.fit_transform(df)
pd.DataFrame(results)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0.882353,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,105.787287,696.563157,185600000.0,3.0,47.0,2.0,0.0,0.0,1.0,0.0
1,0.882353,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,109.202905,445.914611,102080000.0,4.0,78.0,4.0,1.0,0.0,1.0,0.0
2,0.882353,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,111.592017,326.762753,68672000.0,3.0,36.0,4.0,1.0,1.0,1.0,0.0
3,0.882353,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,111.509411,353.257030,69600000.0,3.0,34.0,3.0,1.0,0.0,1.0,0.0
4,0.882353,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,108.692133,511.613629,111360000.0,2.0,33.0,4.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,-0.588235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,114.776517,116.303487,49608000.0,3.0,134.0,1.0,0.0,1.0,0.0,0.0
6447,-1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,119.119719,87.817729,27072000.0,2.0,60.0,2.0,1.0,0.0,0.0,0.0
6448,0.352941,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000000,0.000000,0.0,2.0,55.0,3.0,0.0,0.0,1.0,0.0
6449,0.705882,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,118.524962,134.590951,24550000.0,2.0,44.0,3.0,0.0,0.0,1.0,0.0


### ColumnTransformer

Para realizar los experimentos, se hizo uso de la función `get_pipeline`, la cual es una función altamente customizable que permite configurar diferentes componentes del pre procesamiento y así retornar un objeto `Pipeline` de manera expedita.

In [ ]:
## Código ColumnTransformer

def get_pipeline(custom = False, vectorizer_features = None, reduction_vectorizer = None, vectorizer = 'bow',
                 selection = None, reduction = None, model = True):
    
    '''
    Función que genera un pipeline en función de los parámetros especificados
    custom: indica si incluir features creadas (bool)
    vectorizer_features: indica strings a procesar, lista de la forma ['genres', 'production_companies', 'credits', 'keywords', 'overview]
    reduction_vectorizer: indica el n de componentes a reducir columnas bow (int)
    vectorizer: vectorizador a usar, bow o tfidf (str)
    selection: cantidad o porcentaje de features a ser conservadas según test F (int, 0 < float < 1)
    reduction: reduccion final de features (int)
    model: modelo a ser fiteado
    '''
    
    steps = [] # pasos del pipeline
    union_steps = [] # pasos del Feature Union (Function Transformer + Column Transformer)
    
    # Features creadas en etapa de Feature Engineering
    if custom:
        union_steps.append(('custom_features', fun_transformer))
    
    big_steps = [
        ('scaler', RobustScaler(), ['budget', 'runtime']),
        #('one hot', OneHotEncoder(handle_unknown = 'ignore'), ['original_language']),
    ]
    
    # Vectorización de features string (genres, keywords, production_companies, credits y overview)
    if vectorizer_features:
        categories_steps = [] 
        
        # obtenemos el vectorizador guardado en diccionario
        vect_dict = vectorizer_dict[vectorizer]
        
        # vectorizamos usando el vectorizador escogido (Bag of Words o TFIDF)
        vectorizer_tuples = [(f'vectorizer_{column}', vect_dict[column], column) for column in vectorizer_features]
        categories_ct = ColumnTransformer(vectorizer_tuples, sparse_threshold = 0)
        
        categories_steps.append(('ct', categories_ct))
        
        # podemos elegir reducir la dimensionalidad de los vectores obtenidos
        if reduction_vectorizer:
            categories_steps.append(('reduction', TruncatedSVD(n_components = reduction_vectorizer)))
        
        categories_pipeline = Pipeline(categories_steps)
        
        big_steps.append(('categories', categories_pipeline, 
                          vectorizer_features))

    big_transformer = ColumnTransformer(big_steps, sparse_threshold = 0)
    union_steps.append(('features', big_transformer))
    steps.append(('union', FeatureUnion(union_steps)))

    # Selección de features usando test F
    if selection:
      # podemos seleccionar top n %
      if selection < 1:
          steps.append(('selection', SelectPercentile(f_classif, percentile = selection * 100)))
      # o simplemente seleccionar las top n features
      elif selection > 1: 
          steps.append(('selection', SelectKBest(f_classif, k = selection)))
    
    # Reducción de dimensionalidad
    if reduction:
        steps.append(('reduction', TruncatedSVD(n_components = reduction)))
    
    # Capa de clasificador, podemos elegir no usarla para precargar los datos para el gridsearch
    if model is not None:
        steps.append(('clf', model))
    
    return Pipeline(steps)

# ejemplo
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'credits', 'overview'], selection = .2, 
                        vectorizer = 'tfidf', reduction_vectorizer = 100, reduction = 30, model = LGBMClassifier(objective = 'multiclass', n_estimators = 100, random_state = 3380))

pipeline

Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('custom_features',
                                                 Pipeline(steps=[('features',
                                                                  FunctionTransformer(func=<function custom_features at 0x7f02b9decef0>)),
                                                                 ('onehot_dates',
                                                                  ColumnTransformer(remainder='passthrough',
                                                                                    sparse_threshold=0,
                                                                                    transformers=[('year',
                                                                                                   RobustScaler(),
                                                                                                   ['year']),
                                                              

In [ ]:
# ejemplo de features generadas
pipeline = get_pipeline(custom = True, # se incluyen las features generadas en feature engineering
                        vectorizer_features = ['genres', 'credits', 'overview'], # se vectoriza genres, credits y overview
                        selection = .9, # se selecciona el 90% de las features
                        vectorizer = 'tfidf', # se usa tfidf para vectorizar
                        reduction_vectorizer = 100, # luego del tfidf, se reduce la dimensionalidad de los datos
                        reduction = 30, # finalmente, se escogen las 30 features con más importancia para el problema
                        model = None)

pd.DataFrame(pipeline.fit_transform(df.drop(columns = ['target', 'label']), df['label']))

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.856054e+08,-48658.961324,18499.018145,-21.314667,-10.141571,0.568174,0.832108,-0.796729,-0.145305,0.111032,...,-0.033202,-0.020797,0.171376,0.249154,0.291621,0.601532,-0.484451,-0.072295,-0.066543,0.096779
1,1.020839e+08,-153736.209101,8269.922964,40.309018,1.127832,0.345995,0.352088,0.432701,0.067108,-0.738419,...,0.065016,-0.313671,-0.576214,-0.425820,-0.221072,-0.054619,0.014664,0.062268,-0.098900,-0.012049
2,6.867501e+07,-157571.911670,4751.232905,11.964737,-6.831987,0.090082,-0.015417,0.809412,0.445923,-0.467208,...,0.143405,-0.378794,-0.577376,-0.459406,-0.196147,-0.007066,0.042762,0.168483,-0.122536,0.060532
3,6.960337e+07,-204337.905245,4145.940428,9.402278,-5.991568,0.131153,-0.093774,0.007960,0.618616,-0.348951,...,0.091737,-0.302444,-0.580349,-0.441014,-0.213391,-0.136662,-0.043631,-0.140795,-0.046031,0.059690
4,1.113646e+08,-218581.348862,8258.813102,-7.610037,-6.713678,-0.952997,0.463064,0.573474,-0.635519,0.492558,...,0.065086,-0.077047,0.109731,0.167311,0.337774,0.170325,0.792383,-0.317092,-0.011412,0.293661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,4.960837e+07,131299.207863,-1302.760537,105.104771,14.429104,-1.571887,0.148305,0.774508,-1.432243,-0.319265,...,-0.490418,0.275303,0.022159,-0.001769,0.030470,-0.100789,-0.061492,-0.120172,0.113337,0.178496
6447,2.707256e+07,23661.187302,-2223.300763,44.770195,-0.012506,-1.131837,0.610017,-0.948242,-0.452537,-0.336608,...,0.184751,-0.175896,-0.106530,0.736990,-0.477926,-0.115631,-0.016756,0.144145,-0.111778,0.164864
6448,2.722649e-05,0.000953,-0.062218,53.835843,11.466680,1.572478,-0.764034,-0.460122,1.050383,-0.086676,...,0.188665,-0.334099,-0.666774,-0.444293,-0.229856,-0.084662,-0.049273,-0.263483,0.033378,0.054830
6449,2.455134e+07,-91852.031462,464.568177,35.827199,-4.544916,-0.767861,-0.800286,-0.492344,0.616156,-0.463319,...,0.113147,-0.512879,0.742743,-0.265182,-0.196410,-0.011846,0.026065,0.156304,-0.233837,-0.276939


A *grosso modo* y para facilitar un poco la comprensión, se distinguen los siguientes pasos en el `Pipeline`:

1. **Pre procesamiento**: Involucra la unión de las features creadas por Feature Engineering con las features:
  * Features creadas (opcional): Como casi todas las features creadas corresponden a binarias, el único procesamiento que se hace es escalar mediante `RobustScaeler` la variable `year` y discretizar con `OneHotEncoder` las variables categóricas `month`, `period`, `week`, `duracion`. 
  * Features originales: Consiste del procesamiento de las features numéricas (budget, runtime) y de texto (genres, keywords, credits, production_companies y overview):
    - Features numéricas (obligatorio): Se usa `RobustScaler` en la features para escalarlas y así asegurar una mejor convergencia del modelo. 
    - Features de texto (opcional): Se vectorizan estas features (no son necesariamente todas, son especificadas en la función por el usuario) mediante *Bag of Words* of *TFIDF* (a especificar por el usuario). Además, se habilita la opción de reducir la dimensionalidad de estas features mediante `TruncatedSVD`.
3. **Selección** (opcional): Selección de features. Puede ser porcentual (mediante `SelectPercentile`) o con un número fijo (`SelectKBest`).
4. **Reducción** (opcional): Reducción de dimensionalidad. Como la mayor cantidad de features son *sparse*, se hace uso del algoritmo `TruncatedSVD`.
5. **Modelo** (opcional): Tras todos los pasos anteriores, se concatena una última capa con el modelo predictivo.


---

## 4. Clasificación

### 4.1 Dummy y Baseline

In [ ]:
## Código Dummy

# instanciamos modelo dummy
dummy_clf = DummyClassifier(strategy = 'stratified')

# obtenemos pipeline básico con modelo dummy
basic_pipeline = get_pipeline(model = dummy_clf)

# print de métricas
train_model(pipeline = basic_pipeline, mode = 'clf')

F1: 0.200
                 precision    recall  f1-score   support

          Mixed       0.23      0.23      0.23       335
Mostly Positive       0.46      0.46      0.46       746
       Negative       0.00      0.00      0.00        44
       Positive       0.29      0.29      0.29       442
  Very Positive       0.02      0.02      0.02        46

       accuracy                           0.34      1613
      macro avg       0.20      0.20      0.20      1613
   weighted avg       0.34      0.34      0.34      1613



In [ ]:
## Código Clasificador

# instanciamos decision tree
dt_clf = DecisionTreeClassifier(random_state = 3380)

# obtenemos pipeline básico con decision tree
basic_pipeline = get_pipeline(model = dt_clf)

# print de métricas
train_model(pipeline = basic_pipeline, mode = 'clf')

F1: 0.261
                 precision    recall  f1-score   support

          Mixed       0.32      0.37      0.34       335
Mostly Positive       0.49      0.50      0.49       746
       Negative       0.00      0.00      0.00        44
       Positive       0.40      0.35      0.37       442
  Very Positive       0.11      0.09      0.10        46

       accuracy                           0.41      1613
      macro avg       0.26      0.26      0.26      1613
   weighted avg       0.40      0.41      0.40      1613



  Se utilizó un clasificador Dummy y un DecisionTree como clasificadores básicos para tenerlos como comparación con el modelo baseline que se definirá a continuación. El resultado del primero es bastante bajo (pues utiliza las frecuencias muestrales para generar una predicción), mientras que el segundo mejora en un 40%. Se observa, además, que las clases `Negative` y `Very Positive`, de las cuales se cuenta con pocos ejemplos, serán un desafío a sortear al optimizar el modelo.

---

### 4.2 Búsqueda del mejor modelo de Clasificación


La metodología que se usará para buscar el mejor modelo será:

1. Fijando un modelo predictivo, probar difentes pre procesamientos hasta llegar a aquel con mejor `F1`.

2. Fijando el pre procesamiento, optimizar los hiperparámetros del clasificador por medio de `HalvingGridSearch`

A continuación, se presentan algunos de los experimentos que hicimos:


#### Optimizando Pre procesamiento

In [ ]:
base_clf = LGBMClassifier(objective = 'multiclass', random_state = 3380) # modelo base para comparar

In [ ]:
# partimos de un pipeline "base"
pipeline = get_pipeline(model = base_clf)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'clf', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('features',
                                ColumnTransformer(sparse_threshold=0,
                                                  transformers=[('scaler',
                                                                 RobustScaler(),
                                                                 ['budget',
                                                                  'runtime'])]))])), ('clf', LGBMClassifier(objective='multiclass', random_state=3380))]

F1: 0.259
                 precision    recall  f1-score   support

          Mixed       0.36      0.17      0.23       335
Mostly Positive       0.49      0.70      0.57       746
       Negative       0.00      0.00      0.00        44
       Positive       0.49      0.42      0.45       442
  Very Positive       0.14      0.02      0.04        46

       accuracy                           0.47      1613
      macro avg       0.30      0.26      0.26      1613
   we

In [ ]:
# agregamos features creadas en feature engineering
pipeline = get_pipeline(custom = True, model = base_clf)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'clf', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f086a8f5290>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], vectorizer = 'bow', model = base_clf)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'clf', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f086a8f5290>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], vectorizer = 'bow', model = base_clf)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'clf', param_under = 0.7, param_over = 0.3, fit_weights = True)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f086a8f5290>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = None, selection = 0.7, vectorizer = 'bow', model = base_clf)

train_model(pipeline, mode = 'clf', param_under = 0.7, param_over = 0.3, fit_weights = True)

F1: 0.327
                 precision    recall  f1-score   support

          Mixed       0.42      0.50      0.46       335
Mostly Positive       0.53      0.56      0.54       746
       Negative       0.50      0.05      0.08        44
       Positive       0.53      0.50      0.52       442
  Very Positive       0.07      0.02      0.03        46

       accuracy                           0.50      1613
      macro avg       0.41      0.33      0.33      1613
   weighted avg       0.50      0.50      0.49      1613



#### Optimizando Clasificador

Habiendo elegido el pre procesamiento óptimo, nos preocupamos ahora de optimizar el clasificador:

In [ ]:
pipeline_features = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = None, selection = 0.7, vectorizer = 'bow', model = None)

X = df.drop(columns = ['label', 'target'])
y = df['label'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, random_state = 3380)

X_train, y_train = resample(X_train, y_train, param_over = 0.3, param_under = 0.7)
X_train = pipeline_features.fit_transform(X_train, y_train)

sample_weight = compute_sample_weight(class_weight = 'balanced', y = y_train)

In [ ]:
pipeline_clf = Pipeline([
                     ('clf', RandomForestClassifier())
                     ])

param_grid = [
              # XGBoost
             # {'clf': [XGBClassifier(random_state = 3380)],
             #  "clf__max_depth": [5, 7],
             #  "clf__learning_rate": [0.01, 0.05, 0.1],
              # "clf__gamma": [0, 0.25, 1],
              # },
              # LightGMB
              {'clf': [LGBMClassifier(objective = 'multiclass', random_state = 3380, verbosity = -1, 
                                      device_type = 'gpu' if torch.cuda.is_available() else 'cpu')],
               'clf__n_estimators': [100, 500, 1000, 2000],
               'clf__learning_rate': [0.01, 0.05, 0.1],
               'clf__num_leaves': [31, 50],
               }
]

grid = HalvingGridSearchCV(pipeline_clf, param_grid, cv = 5, verbose = 1, scoring = 'f1_macro', n_jobs = -1)

In [ ]:
model = grid.fit(X_train, y_train, clf__sample_weight = sample_weight)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 411
max_resources_: 3701
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 24
n_resources: 411
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 8
n_resources: 1233
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 2
n_candidates: 3
n_resources: 3699
Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [ ]:
ranking = pd.DataFrame(model.cv_results_)
ranking = ranking.sort_values('rank_test_score')
ranking[[col for col in ranking.columns if 'param' in col.split('_')] + ['mean_train_score', 'mean_test_score']]

,param_clf,param_clf__learning_rate,param_clf__n_estimators,param_clf__num_leaves,mean_train_score,mean_test_score
33,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,2000,31,1.000000,0.520413
34,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,1000,50,1.000000,0.514613
32,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,2000,50,1.000000,0.504287
27,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,1000,50,1.000000,0.371933
24,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,2000,31,1.000000,0.370823
25,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,2000,50,1.000000,0.368030
26,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,1000,31,1.000000,0.365933
31,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,100,50,0.969094,0.358109
28,"LGBMClassifier(device_type='cpu', learning_rat...",0.01,500,50,0.970690,0.356991
30,"LGBMClassifier(device_type='cpu', learning_rat...",0.05,100,31,0.957269,0.349949


In [ ]:
best_params_clf = {key.split('__')[1]: value for key, value in model.best_params_.items() if len(key.split('__')) > 1}
model.best_params_

{'clf': LGBMClassifier(device_type='cpu', learning_rate=0.05, n_estimators=2000,
                objective='multiclass', random_state=3380, verbosity=-1),
 'clf__learning_rate': 0.05,
 'clf__n_estimators': 2000,
 'clf__num_leaves': 31}

In [ ]:
X_test = pipeline_features.transform(X_test)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, average = 'macro') # f1 obtenido en test

0.3254364811036822

Finalmente, el modelo con mejor score lo obtiene LGBMClassifier con parámetros `learning_rate=0.05`, `n_estimators=2000` y `num_leaves=32`. Además, se aprecia que se obtiene un mejor `f1` que el baseline (0.2), el `DecisionTree` (0.26) y el sugerido por el equipo docente en u-cursos :p (0.3)

Entrenamos el modelo una ultima vez con todos los datos:

In [ ]:
model_clf = LGBMClassifier(objective = 'multiclass', random_state = 3380, verbosity = -1,
                           device_type = 'gpu' if torch.cuda.is_available() else 'cpu',
                           **best_params_clf)

pipeline_clf = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = None, selection = 0.7, vectorizer = 'bow', model = model_clf)

# ultimo entrenamiento con toda la data
sample_weight = compute_sample_weight(class_weight = 'balanced', y = y)
pipeline_clf.fit(X, y, clf__sample_weight = sample_weight)

Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('custom_features',
                                                 Pipeline(steps=[('features',
                                                                  FunctionTransformer(func=<function custom_features at 0x7f086a8f5290>)),
                                                                 ('onehot_dates',
                                                                  ColumnTransformer(remainder='passthrough',
                                                                                    sparse_threshold=0,
                                                                                    transformers=[('year',
                                                                                                   RobustScaler(),
                                                                                                   ['year']),
                                                              

---

## 5. Regresión

### 5.1 Dummy y Baseline

In [ ]:
## Código Dummy

dummy_clf = DummyRegressor(strategy = 'mean')

basic_pipeline = get_pipeline(model = dummy_clf)

train_model(pipeline = basic_pipeline, mode = 'reg')

R2: -0.000


In [ ]:
## Código Regresor

from sklearn.tree import DecisionTreeRegressor

# instanciamos random forest
dt_reg = DecisionTreeRegressor(random_state = 3380)

# obtenemos pipeline basico con random forest
basic_pipeline = get_pipeline(model = dt_reg)

# print de métricas
train_model(pipeline = basic_pipeline, mode = 'reg')

R2: 0.194


Se utilizó un regresor Dummy y un DecisionTree como regresores básicos para tenerlos como comparación con el modelo baseline que se definirá a continuación; similar al caso de clasificación. El resultado del primero es muy malo, mientras que el segundo muestra un r2 superior que de todas formas se puede considerar bajo. De esta manera, el desafío será optimizar el modelo para superar, al menos, la barrera de la competencia en Codalab (`R2` = 0.81).

En paralelo, es interesante notar que el `R2` del modelo dummy es 0 puesto que el `R2` esta construido sobre la media $\bar y$ de la variable a predecir. Como usamos la misma media para generar la predicción, es inmediato que el R2 sea 0.

---

### 5.2 Búsqueda del mejor modelo de Regresión


Similar al problema de Clasificación, repetimos la metodología en el problema de Regresión, es decir: 

1. Fijando un modelo predictor, probar difentes pre procesamientos hasta llegar a aquel con mejor `R2`.

2. Fijando el pre procesamiento, optimizar los hiperparámetros del regresor por medio de `HalvingGridSearch`

A continuación, se presentan algunos de los experimentos que hicimos:

#### Optimizando Pre procesamiento

In [ ]:
# Vectorizador BoW
bow_credits = CountVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 4), min_df = 5)
bow_genres = CountVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 3), min_df = 5)
bow_companies = CountVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 3), min_df = 5)
bow_keywords = CountVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 2), min_df = 5, max_df = .8)
bow_overview = CountVectorizer(tokenizer = StemmerTokenizer(), ngram_range = (1, 3), min_df = 5, max_df = .8)

bow_dict = {'credits': bow_credits, 'genres': bow_genres, 
            'production_companies': bow_companies, 'keywords': bow_keywords, 'overview': bow_overview}

# Vectorizador TFIDF
tfidf_credits = TfidfVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 4), min_df = 5)
tfidf_genres = TfidfVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 3), min_df = 5)
tfidf_companies = TfidfVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 3), min_df = 5)
tfidf_keywords = TfidfVectorizer(tokenizer = custom_tokenizer, ngram_range = (1, 2), min_df = 5, max_df = .8)
tfidf_overview = TfidfVectorizer(tokenizer = StemmerTokenizer(), ngram_range = (1, 3), min_df = 5, max_df = .8)

tfidf_dict = {'credits': tfidf_credits, 'genres': tfidf_genres, 
            'production_companies': tfidf_companies, 'keywords': tfidf_keywords, 'overview': tfidf_overview}

# agrupamos ambos en un diccionario
vectorizer_dict = {'bow': bow_dict, 'tfidf': tfidf_dict}

In [ ]:
base_reg = LGBMRegressor(random_state = 3380) # modelo base para comparar

In [ ]:
# partimos de un pipeline "base"
pipeline = get_pipeline(model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg')

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('features',
                                ColumnTransformer(sparse_threshold=0,
                                                  transformers=[('scaler',
                                                                 RobustScaler(),
                                                                 ['budget',
                                                                  'runtime'])]))])), ('clf', LGBMRegressor(random_state=3380))]

R2: 0.531


In [ ]:
# agregamos features creadas en feature engineering
pipeline = get_pipeline(custom = True, model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg')

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7fb9a9c92710>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
# agregamos features de texto
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], vectorizer = 'bow', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg')

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7fb9a9c92710>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
# agregamos features creadas en feature engineering
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = True, selection = 0.7, vectorizer = 'bow', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg')

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7fb9a9c92710>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

Desde acá empezamos a iterar con muchas combinaciones...

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = 200, selection = 0.9, vectorizer = 'bow', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg')

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7fb9a9c92710>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = False, selection = 0.9, vectorizer = 'bow', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg')

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7fb9a9c92710>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = False, selection = 0.9, vectorizer = 'tfidf', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f919036f200>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = 30, selection = 0.9, vectorizer = 'tfidf', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f919036f200>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = 80, reduction = 50, selection = 0.9, vectorizer = 'tfidf', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f919036f200>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = 500, reduction = 300, selection = 0.9, vectorizer = 'tfidf', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f919036f200>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = 500, reduction = 300, selection = 0.9, vectorizer = 'tfidf', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f919036f200>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

In [ ]:
# agregamos features creadas en feature engineering
pipeline = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = 500, selection = 0.9, vectorizer = 'tfidf', model = base_reg)

print(f"Pasos pipeline: {pipeline.steps}" + '\n')

train_model(pipeline, mode = 'reg', param_under = None, param_over = None, fit_weights = False)

Pasos pipeline: [('union', FeatureUnion(transformer_list=[('custom_features',
                                Pipeline(steps=[('features',
                                                 FunctionTransformer(func=<function custom_features at 0x7f919036f200>)),
                                                ('onehot_dates',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   sparse_threshold=0,
                                                                   transformers=[('year',
                                                                                  RobustScaler(),
                                                                                  ['year']),
                                                                                 ('month',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
 

#### Optimizando Regresor

Habiendo elegido el pre procesamiento óptimo, nos preocupamos ahora de optimizar el regresor:

In [ ]:
# agregamos features creadas en feature engineering
pipeline_features = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = False, selection = 0.9, vectorizer = 'tfidf', model = None)

X = df.drop(columns = ['label', 'target'])
y = df['target'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 3380)
X_train = pipeline_features.fit_transform(X_train, y_train)

In [ ]:
pipeline_reg = Pipeline([
                     ('reg', LGBMRegressor())
                     ])

param_grid = [
              # XGBoost
              {'reg': [XGBRegressor(objective = 'reg:squarederror', random_state = 3380)],
               "reg__booster": ['gbtree', 'dart'],
               "reg__eta": [0.01, 0.1],
               "reg__max_depth": [2, 3, 4],
               },
              # LightGMB
              {'reg': [LGBMRegressor(random_state = 3380, verbosity = -1, 
                                      device_type = 'gpu' if torch.cuda.is_available() else 'cpu')],
               'reg__n_estimators': [250, 500, 1000],
               'reg__learning_rate': [0.005, 0.01, 0.05],
               'reg__num_leaves': [20, 31, 50],
               }
]

grid = HalvingGridSearchCV(pipeline_reg, param_grid, cv = 5, verbose = 1, scoring = 'r2', n_jobs = -1)

In [ ]:
model = grid.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 191
max_resources_: 5160
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 39
n_resources: 191
Fitting 5 folds for each of 39 candidates, totalling 195 fits
----------
iter: 1
n_candidates: 13
n_resources: 573
Fitting 5 folds for each of 13 candidates, totalling 65 fits
----------
iter: 2
n_candidates: 5
n_resources: 1719
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 5157
Fitting 5 folds for each of 2 candidates, totalling 10 fits


In [ ]:
ranking = pd.DataFrame(model.cv_results_)
ranking = ranking.sort_values('rank_test_score')
ranking[[col for col in ranking.columns if 'param' in col.split('_')] + ['mean_train_score', 'mean_test_score']].head(30)

,param_reg,param_reg__booster,param_reg__eta,param_reg__max_depth,param_reg__learning_rate,param_reg__n_estimators,param_reg__num_leaves,mean_train_score,mean_test_score
58,"XGBRegressor(eta=0.1, max_depth=4, objective='...",gbtree,0.01,4,NaN,NaN,NaN,0.862900,0.602657
57,"XGBRegressor(eta=0.1, max_depth=4, objective='...",gbtree,0.1,4,NaN,NaN,NaN,0.862900,0.602657
56,"XGBRegressor(eta=0.1, max_depth=4, objective='...",gbtree,0.01,4,NaN,NaN,NaN,0.911162,0.578470
55,"XGBRegressor(eta=0.1, max_depth=4, objective='...",gbtree,0.1,4,NaN,NaN,NaN,0.911162,0.578470
53,"XGBRegressor(eta=0.1, max_depth=4, objective='...",dart,0.01,4,NaN,NaN,NaN,0.911162,0.578470
54,"XGBRegressor(eta=0.1, max_depth=4, objective='...",dart,0.1,4,NaN,NaN,NaN,0.911162,0.578470
52,"XGBRegressor(eta=0.1, max_depth=4, objective='...",dart,0.1,3,NaN,NaN,NaN,0.869813,0.568346
49,"XGBRegressor(eta=0.1, max_depth=4, objective='...",gbtree,0.01,4,NaN,NaN,NaN,0.975015,0.544527
48,"XGBRegressor(eta=0.1, max_depth=4, objective='...",gbtree,0.1,4,NaN,NaN,NaN,0.975015,0.544527
51,"XGBRegressor(eta=0.1, max_depth=4, objective='...",dart,0.1,4,NaN,NaN,NaN,0.975015,0.544527


In [ ]:
best_params_reg = {key.split('__')[1]: value for key, value in model.best_params_.items() if len(key.split('__')) > 1}
model.best_params_

{'reg': XGBRegressor(eta=0.1, max_depth=4, objective='reg:squarederror',
              random_state=3380),
 'reg__booster': 'gbtree',
 'reg__eta': 0.1,
 'reg__max_depth': 4}

In [ ]:
X_test = pipeline_features.transform(X_test)
y_pred = model.predict(X_test)
r2_score(y_test, y_pred) # r2 obtenido en test

0.6121376225012974

Finalmente, el modelo con mejor score lo obtiene XGBRegressor con parámetros `max_depth=4`, `eta=0.1`. Además, se aprecia que se obtiene un mejor `R2` que el baseline (0), el `DecisionTree` (0.194) y el sugerido por el equipo docente en u-cursos :p (0.45)

Entrenamos el modelo una ultima vez con todos los datos:

In [ ]:
model_reg = XGBRegressor(random_state = 3380, **best_params_reg)

pipeline_reg = get_pipeline(custom = True, vectorizer_features = ['genres', 'production_companies', 'keywords', 'credits'], 
                        reduction_vectorizer = False, selection = 0.9, vectorizer = 'bow', model = model_reg)

# ultimo entrenamiento con toda la data
pipeline_reg.fit(X, y)

Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('custom_features',
                                                 Pipeline(steps=[('features',
                                                                  FunctionTransformer(func=<function custom_features at 0x7f087d3b7200>)),
                                                                 ('onehot_dates',
                                                                  ColumnTransformer(remainder='passthrough',
                                                                                    sparse_threshold=0,
                                                                                    transformers=[('year',
                                                                                                   RobustScaler(),
                                                                                                   ['year']),
                                                              

## Generación de Archivo Submit de la Competencia

In [ ]:
from zipfile import ZipFile
import os

def generateFiles(predict_data, clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict(predict_data)
    y_pred_rgr = rgr_pipe.predict(predict_data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")

In [ ]:
# Ejecutar función para generar el archivo de predicciones.
# perdict_data debe tener cargada los datos del text.pickle
# mientras que clf_pipe y rgr_pipe, son los pipeline de 
# clasificación y regresión respectivamente.

!pip3 install pickle5
import pickle5 as pickle

with open(path + 'test.pickle', "rb") as fh:
  predict_data = pickle.load(fh)

generateFiles(predict_data, rgr_pipe = pipeline_reg, clf_pipe = pipeline_clf)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


---

## 6. Conclusiones

A lo largo de todo el trabajo hemos visto como ciertas configuraciones han demostrado ser bastante superiores que otras alternativas. En primer lugar, vimos como la incorporación de features creadas por **Feature Engineering** logra mejorar mucho el pdoer de predicción de los modelos. También vimos como el correcto procesamiento de las variables de texto (`credits`, `genres`, `keywords` y `production_companies`) a través de Bag of Words o TFIDF puede generar una mejor representación de los datos. Además, a pesar de que el set de datos está desbalanceado, metodologías de *resampling* lograron mejoras poco significativas sobre el poder de predicción de los modelos. Finalmente, vimos como el uso de modelos `LightGBM` y `XGBoost` logran superar con bastante facilidad el baseline propuesto.

En términos generales, se concluye que se resuelve exitosamente el problema, logrando tanto para el problema de clasificación como el de regresión métricas superiores a los primeros modelos usando `DecisionTree` (`R2`: 0.19, `F1`: 0.26) y las propuestas por el equipo docente (`R2`: 0.42, `F1`: 0.3). A pesar de lo anterior, se observa que el modelo tiene un desempeño distinto en Codalab, lo que indica que los datos de entrenamiento no son representativos de los datos de test. En ese sentido, si bien se logra obtener un `F1` excelente el Codalab (0.94, bastante superior al 0.77 logrado por el modelo del equipo docente), el modelo de regresión obtiene un `R2` ligeramente más bajo (0.74 vs 0.81). De nuevo, esto puede ser explicado en que la distribución de test no es similar a la distribución de los datos de entrenamiento. En vista de todo lo anterior, se obtienen resultados aceptables.

Finalmente y respecto a los resultados obtenidos, como equipos quedamos un poco frustrados con el desempeño de los modelos :(, sobretodo en el problema de regresión donde no pudimos superar el rendimiento del equipo docente en Codalab. Fueron bastaaaaantes horas que le dedicamos a optimizar el Pipeline del modelo sin mayor éxito, lo cual nos hace inferir que, muy probablemente, los datos de test sean distintos a los de entrenamiento. Una prueba de esto son los puntajes obtenidos en Codalab, los cuales son muy superiores a los obtenidos en la separación interna hecha por nosotros. Así, si los datos de entrenamiento son representativos del problema, lo normal es esperar métricas iguales o inferiores al conjunto de validación, siendo muuy raro el caso donde estas sean mayores. En ese sentido, el principal aprendizaje del trabajo realizado fue dimensionar la importancia de la representatividad de los datos: si no se poseen datos representativos del problema, trabajar el modelo mediante técnicas como ajuste de hiperparametros o la optimización del procesamiento ya no es tan importante, pues el vector objetivo está siendo determinado por componentes que no estan presentes en el conjunto de entrenamiento. Finalmente, quedamos al debe con el uso de la librería `Optuna`, puesto que al tener problemas con la memoria RAM no la pudimos implementar al 100% c: .



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>